In [ ]:
#attempt to improve the NN
#add the local and the global view construct coz transit false positive mismatch seems to be a major problem
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#network and training parameters:
BATCH_SIZE = 128
VERBOSE = 1 #no idea what this is but lets see...
VAL_SPLIT = 0.2  #how much of sample is reserved for validation

In [ ]:
Xtrain=np.loadtxt('Xtrain.csv',delimiter=',')
Ytrain=np.loadtxt('Ytrain.csv',delimiter=',')
Xtest=np.loadtxt('Xtest.csv',delimiter=',')
Ytest=np.loadtxt('Ytest.csv',delimiter=',')
print(Xtrain.shape,Ytrain.shape,Xtest.shape,Ytest.shape)

In [ ]:
#got the training set ... now write the neural net
#buid the model
#SGD stochastic gradient descent... 
model = tf.keras.models.Sequential()
DROPOUT=0.3
IP=5700
TEST=600
np.random.seed(11223)

arr=np.arange(0,IP,1)
np.random.shuffle(arr)
Xtrain=[Xtrain[i] for i in arr]
Ytrain=[Ytrain[i] for i in arr]

arr=np.arange(0,TEST,1)
np.random.shuffle(arr)
Xtest=[Xtest[i] for i in arr]
Ytest=[Ytest[i] for i in arr]

#Xtrain_new=np.array(Xtrain).reshape(5700,500,4)
#print(Xtrain_new.shape)

#add hidden layers
model.add(keras.layers.Reshape((2000, 1), input_shape=(2000,)))
model.add(keras.layers.Conv1D(16,kernel_size=5,strides=2,activation='relu'))
model.add(keras.layers.Conv1D(16,kernel_size=5,strides=2,activation='relu'))
model.add(keras.layers.MaxPool1D(3,strides=3,data_format='channels_last'))
model.add(keras.layers.Conv1D(32,kernel_size=5,strides=2,activation='relu'))
model.add(keras.layers.Conv1D(32,kernel_size=5,strides=1,activation='relu'))
model.add(keras.layers.MaxPool1D(3,strides=3,data_format='channels_last'))
model.add(keras.layers.Conv1D(64,kernel_size=5,strides=1,activation='relu'))
model.add(keras.layers.MaxPool1D(3,strides=3,data_format='channels_last'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(16,name='dense_layer_3',activation='relu'))
model.add(keras.layers.Dense(3,name='dense_layer_4',activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history=model.fit(np.array(Xtrain),np.array(Ytrain), batch_size=64, epochs=100, verbose=VERBOSE, validation_split=VAL_SPLIT, callbacks=[es_callback])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(np.array(Xtest), np.array(Ytest))
print('Test accuracy:', test_acc)
Ypred_raw=model.predict(np.array(Xtrain))
Ypred=np.argmax(Ypred_raw, axis=1)
Ytest_new=np.argmax(Ytrain,axis=1)
cm = confusion_matrix(Ytest_new, Ypred)
print(cm)